# Maestría en Ciencia de Datos, ITAM

**Curso de Optimización 2 2021-1,"Optimización Avanzada"
Prof. Erick Palacios Moreno**

>*Equipo 5:  
MIGUEL LOPEZ  
CARLOS LOPEZ  
JOSÉ ZARATE  

**PERFILAMIENTO DE CÓDIGO.**


EXPLICAR LA REIMPLEMENTACIÓN QUE SE DESARROLLO 

El paquete que desarrollamos se compone de dos métodos de optimización convexa, uno metaheurístico Hill Climbing y uno por descenso en gradiente Conjugate Gradient. En este notebook realizaremos el perfilamiento de tales métodos, Hill Climbing aplicado para resolver el problema del viajero (TSP) por medio del análisis de la ruta más corta por distancia euclideana.

Nota: el desarrollo de este notebook está basado en la secuencia de perfilamiento utilizada por el Profesor Erick Palacios Moreno en sus notas de clase (ver referencias en README).

-----

## RESOLVER EL PROBLEMA DE TRAVELING SALESMAN PROBLEM (A.K.A. TSP) PARA DISTINTAS CIUDADES UTILIZANDO EL MÉTODO DE HILL CLIMBING

* Distancia Euclideana.
> Dataset: National Traveling Salesman Problems, CANADA

Para ello, realizaremos una ejecución para segemento del dataset con 100 ciudades:

______

**Hill Climbing**  

.

______

**MODIFICACIONES DERIVADAS DE LA ETAPA DE EXPERIMENTACIÓN:**

.

-----

**PERFILAMIENTO**  

> Descripción de la instancia

Hemos decidido utilizar una máquina AWS EC2 m5.2xlarge que es la de mayor capacidad disponible con el programa AWS EDUCATE. Sus características son:

In [13]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              2
Core(s) per socket:              4
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz
Stepping:                        4
CPU MHz:                         3113.072
BogoMIPS:                        5000.00
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       128 KiB
L1i cache:                       128 KiB
L2 cache:                        4 MiB
L3 cache:                        33 MiB
NUMA node0 CPU(s):               0-7

In [14]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Amazon EC2
       physical id: 0
       version: 1.0
       date: 10/16/2017
       size: 64KiB
       capacity: 64KiB
       capabilities: pci edd acpi virtualmachine
  *-memory
       description: System memory
       physical id: 1
       size: 31GiB


Como podemos observar, tenemos disponibles 8 cores 30 gigas de memoria.

Ahora mostraremos las características del kernel utilizado.

In [15]:
%%bash
uname -ar #r for kernel, a for all

Linux ip-10-0-0-123 5.4.0-1047-aws #49-Ubuntu SMP Wed Apr 28 22:47:04 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


* **Nota:** Como bien sabemos, no es una buena práctica realizar el análisis de perfilamiento de código de nuestros métodos o paquetes desarrollados utilizando _Jupyter notebooks_ debido a que se encapsulan procesos propios del _Jupyter_ y no aisla específicamente los procesos ejecutados del código perfilado. La sugerencia es realizarlo en un intérprete (nuestro caso de _Python_). Por temas académicos, en esta ocasión se desarrollará en el notebook.

----

> **Medición de tiempo**


En primer término evaluaremos el tiempo de ejecución que le toma a nuestro código desarrollado en práctica 2.1 con las reimplementaciones realizadas previamente.

Importaremos nuestro paquete alojado en [Dockerhub](https://hub.docker.com) en la imágen lobolc/Opt_HC_CG:0.1. Para ello utilizaremos el repositorio de desarollo de GitHub de nuestra practica anterior.
**NOTA PARA MI** Definir si el párrafo anterior se indica sólo en este punto o en un README.



In [4]:
# Import our Package to profile
# Nota: en este caso utilizaremos las funciones integradas en el módulo hill_final.py que contiene la
# reimplementación que fue necesario realizar sobre nuestro método.

import hill_final as hc

In [5]:
# Packages and libraries required

import math
import time
import numpy as np
import pandas as pd

In [6]:
# Testing packages requirements
#!pip install --upgrade --user ortools

from pytest import approx
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

Extraeremos y aplicaremos algunas transformaciones a nuestro dataset:

In [ ]:
raw_data = pd.read_csv("../datasets/ca4663.tsp", sep = " ", names = ['index','x','y'])

Para efectos de esta práctica, analizaremos un ejemplo didáctico práctico con 10 ciudades

In [11]:
raw_data1 = raw_data.drop(['index'], axis = 1)
raw_data2 = raw_data1.dropna()
tsp_cities = raw_data2.iloc[0:10,].to_numpy()

En nuestra función _opt.best_solution()_ desarrollamos el método de _Hill Climbing_ para la determinación de la ruta óptima y la distancia menor al resolver el problema de _Traveling Salesman Problem_ (TSP). También se incluyo la función time del paquete time time que se utiliza en el perfilamiento. Está constituida por cuatro subfunciones a las que se llama durante la ejecución:

best_solution(coordinate, initial_point = 0, tolerance = 1e-7)
neighbors(matrix, solution)
calculate_distance(matrix, solution)
random_solution(matrix, initial_point)
distance_matrix(coordinate)

In [9]:
# parameters; dataset, initial point, tolerance and number of restart

best_distance, best_path, exec_time = hc.best_solution(tsp_cities, 0 , 1e-6, 100)


KeyboardInterrupt: 

In [62]:
print(best_distance)
print(best_path)
print(exec_time)

16310.936025236724
[0, 12, 39, 48, 45, 44, 49, 47, 41, 38, 31, 32, 21, 8, 10, 36, 43, 46, 29, 11, 13, 24, 35, 33, 34, 40, 42, 37, 26, 5, 3, 4, 2, 7, 9, 17, 18, 22, 19, 20, 23, 16, 14, 15, 28, 30, 27, 25, 6, 1]
0.9718306064605713


In [ ]:

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(tsp_cities), 1, 0)

# Create the routing index manager.
routing = pywrapcp.RoutingModel(manager)

# Create Routing Model.
distance_matrix = compute_euclidean_distance_matrix(tsp_cities)

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(manager, routing, solution)


In [ ]:
compute_euclidean_distance_matrix(tsp_cities)

In [ ]:
opt.distance_matrix(tsp_cities)